In [1044]:
import pyomo.environ as pyo
from pyomo.dae import ContinuousSet, DerivativeVar
import numpy as np
import pandas as pd

In [1045]:
# Define model
model = pyo.ConcreteModel()

In [1046]:
# Global parameters
model.R = pyo.Param(initialize=0.082057) # L·atm/(mol·K)

In [1047]:
# Reator Configuration
L = 50  # cm
ID = 0.052  # cm
nx = 11
A = np.pi * ID**2 / 4

model.L = pyo.Param(initialize=L)
model.ID = pyo.Param(initialize=ID)
model.nx = pyo.Param(initialize=nx)
model.A = pyo.Param(initialize=A)

In [1048]:
# Domain
model.x = ContinuousSet(bounds=(0, L), initialize=np.linspace(0, L, nx))

In [1049]:
# Component properties
comp = ['CO2', 'H2', 'CH4', 'CO', 'H2O']
cp = {'CO2': 37.1, 'H2': 28.8, 'CH4': 35.3, 'CO': 29.1, 'H2O': 33.6}

In [1050]:
# Feed conditions
R = 0.082057  # L·atm/(mol·K)
P = 1  # atm
T0 = 273.15  # K
Fvi = 100  # cm3/min
Fi = P*Fvi/1000/60/R/T0  # mol/s
Ti = 473.15  # K
zi_values = {'CO2': 0.2, 'H2':0.8, 'CH4':0.0, 'CO':0.0, 'H2O':0.0}
rhoi = P/R/Ti  # mol/L
Fv = Fi/rhoi  # L/s
V0 = Fv*1000/A  # cm/s

model.P = pyo.Param(initialize=P)
model.T0 = pyo.Param(initialize=T0)
model.Fvi = pyo.Param(initialize=Fvi)
model.Fi = pyo.Param(initialize=Fi)
model.Ti = pyo.Param(initialize=Ti)
model.zi = pyo.Param(comp, initialize=zi_values)
model.rhoi = pyo.Param(initialize=rhoi)
model.Fv = pyo.Param(initialize=Fv)
model.V0 = pyo.Param(initialize=V0)
model.Ci = pyo.Param(comp, initialize={c: rhoi*zi_values[c] for c in comp})
model.cp = pyo.Param(comp, initialize=cp)

In [1051]:
# source term
Q = 10000 # W
k0 = 2e0
k1 = 1e0
k2 = 1e0
hrx1 = 41000 # J/mol
hrx2 = -165000 # J/mol
hrx3 = 247300 # J/mol

# Thermodynamics

# Initial Conditions
def C_init(model, x, c):
    return model.Ci[c]

def z_init(model, x, c):
    return model.zi[c]

def T_init(model, x):
    return model.Ti

def cpx_init(model, x):
    total = 0
    for ci in comp:
        total += model.cp[ci] * model.zi[ci]
    return total

def P_init(model, x ,c):
    return model.P * model.zi[c]


model.C = pyo.Var(model.x, comp, domain=pyo.NonNegativeReals, initialize=C_init)
model.dCdx = DerivativeVar(model.C, wrt=model.x)
model.rho = pyo.Var(model.x, domain=pyo.NonNegativeReals, initialize=model.rhoi)
model.drhodx = DerivativeVar(model.rho, wrt=model.x)
model.z = pyo.Var(model.x, comp, domain=pyo.NonNegativeReals, initialize=z_init)
model.V = pyo.Var(model.x, domain=pyo.NonNegativeReals, initialize=model.V0)
model.dVdx = DerivativeVar(model.V, wrt=model.x)
model.T = pyo.Var(model.x, domain=pyo.NonNegativeReals, initialize=T_init)
model.dTdx = DerivativeVar(model.T, wrt=model.x)
model.cpx = pyo.Var(model.x, domain=pyo.NonNegativeReals, initialize=cpx_init)
model.pP = pyo.Var(model.x,comp,domain=pyo.NonNegativeReals,initialize=P_init)
model.r1 = pyo.Var(model.x,domain=pyo.NonNegativeReals,initialize=0) # RWGS: CO2 + H2 => CO + H2O
model.r2 = pyo.Var(model.x,domain=pyo.NonNegativeReals,initialize=0) # Methanation: CO2 + 4H2 => CH4 + 2H2O
model.r3 = pyo.Var(model.x,domain=pyo.NonNegativeReals,initialize=0) # Dry reforming: CH4 + CO2 => 2CO + 2H2

# Algebraic Equations
def cal_rho(model, x):
    return model.rho[x] == model.P / (model.R * model.T[x])
model.rho_constr = pyo.Constraint(model.x, rule=cal_rho)

def cal_z(model, x, c):
    return model.z[x, c] == model.C[x, c] / model.rho[x]
model.z_constr = pyo.Constraint(model.x, comp, rule=cal_z)

def cal_cpx(model, x):
    return model.cpx[x] == sum(model.cp[c] * model.z[x, c] for c in comp)
model.cpx_constr = pyo.Constraint(model.x, rule=cal_cpx)

def cal_r1(model, x):
    return model.r1[x] == k0 * model.pP[x, 'CO2'] * model.pP[x, 'H2']
model.r1_constr = pyo.Constraint(model.x, rule=cal_r1)

def cal_r2(model, x):
    return model.r2[x] == k1 * model.pP[x, 'CO2'] * model.pP[x, 'H2']**4
model.r2_constr = pyo.Constraint(model.x, rule=cal_r2)

def cal_r3(model, x):
    return model.r3[x] == k2 * model.pP[x, 'CH4'] * model.pP[x, 'CO2']
model.r3_constr = pyo.Constraint(model.x, rule=cal_r3)

def cal_pP(model, x, c):
    return model.pP[x, c] == model.P * model.z[x, c]
model.pP_constr = pyo.Constraint(model.x, comp, rule=cal_pP)

# Balance Equations
def continuity(model, x):
    return model.rho[x] * model.dVdx[x] + model.drhodx[x] * model.V[x] == 0
model.continuity_constr = pyo.Constraint(model.x, rule=continuity)

def component_balance(model, x, c):
    if c == 'CO2':
        return model.V[x] * model.dCdx[x, c] + model.C[x, c] * model.dVdx[x] == -model.r1[x] - model.r2[x] - model.r3[x]
    elif c == 'H2':
        return model.V[x] * model.dCdx[x, c] + model.C[x, c] * model.dVdx[x] == -model.r1[x] - 4*model.r2[x] + 2*model.r3[x]
    elif c == 'CH4':
        return model.V[x] * model.dCdx[x, c] + model.C[x, c] * model.dVdx[x] == model.r2[x] - model.r3[x]
    elif c == 'CO':
        return model.V[x] * model.dCdx[x, c] + model.C[x, c] * model.dVdx[x] == model.r1[x] + 2*model.r3[x]
    elif c == 'H2O':
        return model.V[x] * model.dCdx[x, c] + model.C[x, c] * model.dVdx[x] == model.r1[x] + 2*model.r2[x]
model.component_balance_constr = pyo.Constraint(model.x, comp, rule=component_balance)

def energy_balance(model, x):
    return model.rho[x] * model.V[x] * model.cpx[x] * model.dTdx[x] == Q - (model.r1[x] * hrx1 + model.r2[x] * hrx2 + model.r3[x] * hrx3)
model.energy_balance_constr = pyo.Constraint(model.x, rule=energy_balance)

# Boundary Conditions
def init_C(model, comp):
    return model.C[0, comp] == model.Ci[comp]
model.init_C_constr = pyo.Constraint(comp, rule=init_C)

def init_V(model):
    return model.V[0] == model.V0
model.init_V_constr = pyo.Constraint(rule=init_V)

def init_T(model):
    return model.T[0] == model.Ti
model.init_T_constr = pyo.Constraint(rule=init_T)

In [1052]:
# Apply finite difference discretization
discretizer = pyo.TransformationFactory('dae.finite_difference')
discretizer.apply_to(model, nfe=nx-1, wrt=model.x, scheme='BACKWARD')

In [1053]:
solver = pyo.SolverFactory('ipopt')
# Add solver options for better numerical stability
solver.options['tol'] = 1e-6  # Relax tolerance slightly
solver.options['dual_inf_tol'] = 1e-4  # Allow higher dual infeasibility
solver.options['constr_viol_tol'] = 1e-6  # Constraint violation tolerance
solver.options['max_iter'] = 3000  # More iterations if needed
solver.options['mu_init'] = 1e-3  # Better initial barrier parameter
solver.options['nlp_scaling_method'] = 'gradient-based'  # Better scaling

result = solver.solve(model, tee=True)

Ipopt 3.14.19: tol=1e-06
dual_inf_tol=0.0001
constr_viol_tol=1e-06
max_iter=3000
mu_init=0.001
nlp_scaling_method=gradient-based


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.19, running with linear solver MUMPS 5.7.3.

Number of nonzeros in equality constraint Jacobian...:     1182
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:      308

Total number of variables............................:      330
                     variables with only lower bounds:      242
                variables with lower and upper bounds:        0
                     varia

In [1054]:
# 모든 x 축에 대해 V, rho, C, z 값을 출력하고, 체계적으로 정렬하여 저장
import re
profile_data = []

# V 값 저장
for i in model.x:
    profile_data.append(['V', i, '', model.V[i].value])

# rho 값 저장
for i in model.x:
    profile_data.append(['rho', i, '', model.rho[i].value])

# T 값 저장
for i in model.x:
    profile_data.append(['T', i, '', model.T[i].value])

# cpx 값 저장
for i in model.x:
    profile_data.append(['cpx', i, '', model.cpx[i].value])

# C[x, c] 값 저장
for c in comp:
    for i in model.x:
        profile_data.append(['C', i, c, model.C[i, c].value])

# r1[x] 값 저장
for i in model.x:
    profile_data.append(['r1', i, '', model.r1[i].value])

# r2[x] 값 저장
for i in model.x:
    profile_data.append(['r2', i, '', model.r2[i].value])

# r3[x] 값 저장
for i in model.x:
    profile_data.append(['r3', i, '', model.r3[i].value])

# pP[x, c] 값 저장
for c in comp:
    for i in model.x:
        profile_data.append(['pP', i, c, model.pP[i, c].value])

# z[x, c] 값 저장
for c in comp:
    for i in model.x:
        profile_data.append(['z', i, c, model.z[i, c].value])

# 정렬: 변수명 -> 성분명 -> x좌표 순 (같은 성분명끼리 묶임)
def sort_key(item):
    var_name, x_coord, component, value = item
    # 변수명 -> 성분명 -> x좌표 순으로 정렬
    return (var_name, component, float(x_coord))

sorted_data = sorted(profile_data, key=sort_key)

# DataFrame 생성
df_profile = pd.DataFrame(sorted_data, columns=['변수명', 'x좌표', '성분명', '값'])

# CSV 저장
df_profile.to_csv('profile.csv', index=False)

print("Profile saved to profile.csv (grouped by component)")
print(df_profile.head(20))

Profile saved to profile.csv (grouped by component)
   변수명   x좌표  성분명             값
0    C   0.0  CH4  0.000000e+00
1    C   5.0  CH4  1.511198e-04
2    C  10.0  CH4  2.323466e-04
3    C  15.0  CH4  2.763308e-04
4    C  20.0  CH4  2.988999e-04
5    C  25.0  CH4  3.086225e-04
6    C  30.0  CH4  3.104770e-04
7    C  35.0  CH4  3.074908e-04
8    C  40.0  CH4  3.015600e-04
9    C  45.0  CH4  2.938948e-04
10   C  50.0  CH4  2.852764e-04
11   C   0.0   CO  4.725396e-34
12   C   5.0   CO  7.595492e-04
13   C  10.0   CO  1.269244e-03
14   C  15.0   CO  1.611849e-03
15   C  20.0   CO  1.837643e-03
16   C  25.0   CO  1.980343e-03
17   C  30.0   CO  2.063597e-03
18   C  35.0   CO  2.104331e-03
19   C  40.0   CO  2.114776e-03
